#日本語分かち書きテキストの作成



1.   収集したテキストを1行1文書（段落など）のテキストファイルとして作成する．このとき，テキストファイルはUTF-8(N)で保存する
2.   1.で作成したテキストファイルを（通常は左側にある）ファイルのアイコンを開いた領域（プログラムからは/content/に見える）にアップロードする
3. 2.でアップロードしたファイル名と分かち書き後のファイルを下記のコードセルに書き込む
4. 下記のコードセルを実行する（↑のメニューにある”ランタイム”から”すべてのセル実行”で実行）
5. 生成されたテキストファイル(*_wakati.txt)をダウンロードする
6. クラスタリングの場合は，そのまま利用する．分類の場合は，分類ラベルの列とテキストの列からなるCSVファイルを作成し，ARFFに編集する



In [ ]:
#Mecabの基本パッケージと開発用パッケージをインストールしておく
!apt -q -y install mecab libmecab-dev file


In [ ]:
# Pythonで品詞を選択しつつ分かち書き（MeCab + UniDic + Neologd）
!pip install mecab-python3
!pip install neologdn
!pip install unidic
!python -m unidic download

# 下記のコマンドを実行（先頭の#を消去して実行）するとNeologdの辞書がインストールされる（かなり時間がかかります）
#!git clone --depth 1 https://github.com/neologd/mecab-unidic-neologd.git
#!echo yes | mecab-unidic-neologd/bin/install-mecab-unidic-neologd -n


In [ ]:
# テキストファイルを開いて，一行ずつ分かち書きを行う
import MeCab
import unidic
import neologdn
import re

txt_file_name = 'test.txt' #アップロードするテキストファイル名にする←※要書き換え
result_file_name = 'test_wakati.txt' #出力する分かち書きされたファイル名←※要書き換え
with open(result_file_name,'w') as f:
  f.write('')
target_pos_tags = ['名詞', '動詞', '形容詞', '形容動詞', '副詞'] #分かち書き後に対象とする品詞
stop_words = ['の','さ'] #間違って形態素解析解析をされるなどして，不要になる単語

tagger = MeCab.Tagger() #UniDicのまま使うのであれば，こちらをつかう
#tagger = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-unidic-neologd') #Neologdを使うのであれば，こちらを使う

#テキストファイルを開いて各行（1文書＝1段落や1見出しなど）
with open(txt_file_name, 'r') as f:
  line = f.readline()
  while line:
    result_line_array = []
    #各行の内容を正規化（内容をきれいにする）
    line = neologdn.normalize(line)
    #HTMLタグを取り除く
    line = re.sub(r"<(.+?)>","",line)
    #URL（のパターン）を取り除く
    line = re.sub(r"(https?|ftp)(:\/\/[-_\.!~*\'()a-zA-Z0-9;\/?:\@&=\+\$,%#]+)", "" ,line)
    parsed_lines = tagger.parse(line).split('\n')
    #print(parsed_lines)
    for token in parsed_lines: #各行を形態素解析
      m_result = str(token).split('\t') #形態素結果を表層形とその他の結果に分割
      face = m_result[0] #表層形を取り出す
      if face == "EOS": break
      face = face.replace(',','，')
      word = face #単語を表層形の単語とする
      #word = m_info_list[6] #ここでfaceの代わりにm_info_list[6]を集計対象とすると”原形”で数え上げが可能になる
      #print(str(m_result[0])+" "+m_result[1]) #←形態素解析結果を出力する場合は先頭のコメントを外す
      m_info_list = m_result[1].split(',') #形態素解析結果（品詞以降）を分割
      m_pos = m_info_list[0] #形態素解析結果の先頭の品詞（大分類）を取り出す(1:中分類、2:小分類)
      #print(face+' '+str(m_info_list))
      if m_pos in target_pos_tags and word not in stop_words: #対象の品詞のリスト内に解析結果の品詞があれば分かち書き対象の単語とする
        result_line_array.append(word)

    #分かち書き結果をファイルに出力する
    with open(result_file_name,'a') as rf:
      rf.write('"'+' '.join(result_line_array)+'"\n') #各行を"で挟むのを忘れずに！

    #テキストファイルの次の行を読み込む
    line = f.readline()
